# Oversampling 6 Time Bellweathers SVM

### Import Libaries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics

### Import & Clean Dataset

In [2]:
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
df = pd.read_csv('D:/MEGAN DATA/ASDRP/Fall2020/Bellwether Analysis/data/segmented_data/data_2004_2020.csv')

In [4]:
df.head()

Unnamed: 0  fips    state   county  wins  poverty_rate     population  \
0           1  1001  Alabama  Autauga     3     21.933333   34419.166667   
1           2  1003  Alabama  Baldwin     3     19.716667  100428.166667   
2           3  1005  Alabama  Barbour     3     36.383333   24581.000000   
3           4  1007  Alabama     Bibb     3     29.083333   17001.000000   
4           5  1009  Alabama   Blount     3     21.583333   39136.666667   

   persons_in_poverty  rural_urban_code  urban_influence_code  metro  \
0         6205.166667                 4                     4      0   
1        16443.333333                 4                     4      0   
2         8902.166667                 4                     4      0   
3         4679.500000                 4                     4      0   
4         7474.666667                 4                     4      0   

   civilian_labor_force  employed  unemployed  unemployment_rate  no_diploma  \
0               29003.5  27208.45     1794.45               5.75     10414.2   
1               29003.5  27208.45     1794.45               5.75      6002.6   
2               29003.5  27208.45     1794.45               5.75     17145.6   
3               29003.5  27208.45     1794.45               5.75      6884.8   
4               29003.5  27208.45     1794.45               5.75      4602.0   

   diploma_only  some_college/associate_degree  \
0       10515.8                         7802.0   
1        7604.8                         5182.8   
2       22671.4                        19807.8   
3        4556.2                         2631.2   
4        3945.8                         1741.6   

   completed_college/bachelor_degree  percent_no_diploma  \
0                             6454.4               31.80   
1                             4234.6               31.60   
2                            17325.4               30.72   
3                             1661.2               46.12   
4                              797.2               46.88   

   percent_diploma_only  percent_some_college/associate_degree  \
0                 24.80                                  14.20   
1                 32.94                                  19.72   
2                 29.88                                  21.42   
3                 28.06                                  15.54   
4                 33.16                                  13.48   

   percent_completed_college/bachelor_degree  percent_white_male  \
0                                       8.20           39.022558   
1                                      14.60           42.978081   
2                                      16.84           27.180683   
3                                       9.90           39.515601   
4                                       6.34           47.796657   

   percent_white_female  percent_black_male  percent_black_female  \
0             40.318118            8.354254              9.400597   
1             44.647080            4.606480              4.870930   
2             23.760790           24.795134             22.336745   
3             37.067423           13.436614              8.644992   
4             48.794529            0.732703              0.692579   

   percent_native_american_male  percent_native_american_female  \
0                      0.218065                        0.254714   
1                      0.363207                        0.371437   
2                      0.338711                        0.265870   
3                      0.152738                        0.144010   
4                      0.310526                        0.301804   

   percent_asian_male  percent_asian_female  percent_hispanic_male  \
0            0.366495              0.520423               1.271738   
1            0.296272              0.454832               2.427784   
2            0.214881              0.193029               2.829879   
3            0.056731              0.039276               1.300458

In [5]:
df = df.drop(['Unnamed: 0'], axis=1)
len(df['wins'])

3108

In [6]:
# Add 2020 bellwether code (here we specify wins to 6)
# Specifying a bellwether to be either 5 or 6 wins would result in 59 bellweathers under the new classification

binary_bellweathers = []
for i, v in enumerate(df['wins']):
    if df.iloc[i, 3] == 5 or df.iloc[i, 3] == 6:
        binary_bellweathers.append(1)
    elif df.iloc[i, 3] < 5:
        binary_bellweathers.append(0)

In [7]:
binary_bellweathers.count(1)

62

In [8]:
# gets rid of irrelevant columns, strings, and wins (since that would give the model 100% accuracy)
df = df.drop(['fips', 'wins'], axis=1)
df = df.drop(['state', 'county'], axis=1)
df.head()

poverty_rate     population  persons_in_poverty  rural_urban_code  \
0     21.933333   34419.166667         6205.166667                 4   
1     19.716667  100428.166667        16443.333333                 4   
2     36.383333   24581.000000         8902.166667                 4   
3     29.083333   17001.000000         4679.500000                 4   
4     21.583333   39136.666667         7474.666667                 4   

   urban_influence_code  metro  civilian_labor_force  employed  unemployed  \
0                     4      0               29003.5  27208.45     1794.45   
1                     4      0               29003.5  27208.45     1794.45   
2                     4      0               29003.5  27208.45     1794.45   
3                     4      0               29003.5  27208.45     1794.45   
4                     4      0               29003.5  27208.45     1794.45   

   unemployment_rate  no_diploma  diploma_only  some_college/associate_degree  \
0               5.75     10414.2       10515.8                         7802.0   
1               5.75      6002.6        7604.8                         5182.8   
2               5.75     17145.6       22671.4                        19807.8   
3               5.75      6884.8        4556.2                         2631.2   
4               5.75      4602.0        3945.8                         1741.6   

   completed_college/bachelor_degree  percent_no_diploma  \
0                             6454.4               31.80   
1                             4234.6               31.60   
2                            17325.4               30.72   
3                             1661.2               46.12   
4                              797.2               46.88   

   percent_diploma_only  percent_some_college/associate_degree  \
0                 24.80                                  14.20   
1                 32.94                                  19.72   
2                 29.88                                  21.42   
3                 28.06                                  15.54   
4                 33.16                                  13.48   

   percent_completed_college/bachelor_degree  percent_white_male  \
0                                       8.20           39.022558   
1                                      14.60           42.978081   
2                                      16.84           27.180683   
3                                       9.90           39.515601   
4                                       6.34           47.796657   

   percent_white_female  percent_black_male  percent_black_female  \
0             40.318118            8.354254              9.400597   
1             44.647080            4.606480              4.870930   
2             23.760790           24.795134             22.336745   
3             37.067423           13.436614              8.644992   
4             48.794529            0.732703              0.692579   

   percent_native_american_male  percent_native_american_female  \
0                      0.218065                        0.254714   
1                      0.363207                        0.371437   
2                      0.338711                        0.265870   
3                      0.152738                        0.144010   
4                      0.310526                        0.301804   

   percent_asian_male  percent_asian_female  percent_hispanic_male  \
0            0.366495              0.520423               1.271738   
1            0.296272              0.454832               2.427784   
2            0.214881              0.193029               2.829879   
3            0.056731              0.039276               1.300458   
4            0.078504              0.139562               4.481700   

   percent_hispanic_female  cumulative_confirmed_cases  cumulative_deaths  
0                 1.128805                      733294              10242  
1                 1.957041                     2279213  

In [9]:
df.insert(0, "bellweather?", binary_bellweathers)

In [10]:
df.head()

bellweather?  poverty_rate     population  persons_in_poverty  \
0             0     21.933333   34419.166667         6205.166667   
1             0     19.716667  100428.166667        16443.333333   
2             0     36.383333   24581.000000         8902.166667   
3             0     29.083333   17001.000000         4679.500000   
4             0     21.583333   39136.666667         7474.666667   

   rural_urban_code  urban_influence_code  metro  civilian_labor_force  \
0                 4                     4      0               29003.5   
1                 4                     4      0               29003.5   
2                 4                     4      0               29003.5   
3                 4                     4      0               29003.5   
4                 4                     4      0               29003.5   

   employed  unemployed  unemployment_rate  no_diploma  diploma_only  \
0  27208.45     1794.45               5.75     10414.2       10515.8   
1  27208.45     1794.45               5.75      6002.6        7604.8   
2  27208.45     1794.45               5.75     17145.6       22671.4   
3  27208.45     1794.45               5.75      6884.8        4556.2   
4  27208.45     1794.45               5.75      4602.0        3945.8   

   some_college/associate_degree  completed_college/bachelor_degree  \
0                         7802.0                             6454.4   
1                         5182.8                             4234.6   
2                        19807.8                            17325.4   
3                         2631.2                             1661.2   
4                         1741.6                              797.2   

   percent_no_diploma  percent_diploma_only  \
0               31.80                 24.80   
1               31.60                 32.94   
2               30.72                 29.88   
3               46.12                 28.06   
4               46.88                 33.16   

   percent_some_college/associate_degree  \
0                                  14.20   
1                                  19.72   
2                                  21.42   
3                                  15.54   
4                                  13.48   

   percent_completed_college/bachelor_degree  percent_white_male  \
0                                       8.20           39.022558   
1                                      14.60           42.978081   
2                                      16.84           27.180683   
3                                       9.90           39.515601   
4                                       6.34           47.796657   

   percent_white_female  percent_black_male  percent_black_female  \
0             40.318118            8.354254              9.400597   
1             44.647080            4.606480              4.870930   
2             23.760790           24.795134             22.336745   
3             37.067423           13.436614              8.644992   
4             48.794529            0.732703              0.692579   

   percent_native_american_male  percent_native_american_female  \
0                      0.218065                        0.254714   
1                      0.363207                        0.371437   
2                      0.338711                        0.265870   
3                      0.152738                        0.144010   
4                      0.310526                        0.301804   

   percent_asian_male  percent_asian_female  percent_hispanic_male  \
0            0.366495              0.520423               1.271738   
1            0.296272              0.454832               2.427784   
2            0.214881              0.193029               2.829879   
3            0.056731              0.039276               1.300458   
4            0.078504              0.139562               4.481700   

   percent_hispanic_female  cumulative_confirmed_cases  cumulative_deaths  
0                

In [11]:
df["bellweather?"].value_counts()

0    3046
1      62
Name: bellweather?, dtype: int64

In [12]:
# only take the 59 bellweathers

df1 = df[df["bellweather?"] == 1]

In [13]:
df1.head()

bellweather?  poverty_rate     population  persons_in_poverty  \
287             1     14.616667  103085.000000        13997.000000   
568             1     12.916667   17383.833333         2279.666667   
603             1     11.500000   22077.500000         2520.500000   
654             1     14.450000   19146.666667         2792.666667   
680             1     15.516667  113672.666667        17493.833333   

     rural_urban_code  urban_influence_code  metro  civilian_labor_force  \
287                 1                     1      1             459611.30   
568                 4                     4      0              29003.50   
603                 9                    12      0               1693.25   
654                 9                     8      0               2716.25   
680                 1                     1      1              56099.60   

      employed  unemployed  unemployment_rate  no_diploma  diploma_only  \
287  432442.40    27168.90              5.875    129100.8      164385.6   
568   27208.45     1794.45              5.750      1708.2        2046.2   
603    1546.15      147.10              8.680      3328.0        5596.4   
654    2544.95      171.30              6.310      1500.6        2108.2   
680   53041.15     3058.45              5.465      6537.2       11282.8   

     some_college/associate_degree  completed_college/bachelor_degree  \
287                       109497.2                           142428.6   
568                         1377.4                              668.0   
603                         3031.2                             1901.8   
654                          968.0                              553.6   
680                         4837.8                             2491.0   

     percent_no_diploma  percent_diploma_only  \
287               24.92                 30.40   
568               31.50                 35.06   
603               23.78                 40.34   
654               29.32                 41.10   
680               26.40                 44.92   

     percent_some_college/associate_degree  \
287                                  19.48   
568                                  22.38   
603                                  22.08   
654                                  18.82   
680                                  18.88   

     percent_completed_college/bachelor_degree  percent_white_male  \
287                                      23.84           33.904257   
568                                       9.36           48.651459   
603                                      12.86           49.193051   
654                                      10.24           46.399729   
680                                       9.02           43.183962   

     percent_white_female  percent_black_male  percent_black_female  \
287             35.593001           11.462633             13.033701   
568             48.761942            0.571911              0.279457   
603             48.999030            0.286621              0.185202   
654             49.539730            1.101259              0.745468   
680             46.553526            3.391660              3.620263   

     percent_native_american_male  percent_native_american_female  \
287                      0.341322                        0.357957   
568                      0.188471                        0.162475   
603                      0.092601                        0.110239   
654                      0.180719                        0.129892   
680                      0.133423                        0.137672   

     percent_asian_male  percent_asian_female  percent_hispanic_male  \
287            0.865011              1.233442               2.821145   
568            0.175473              0.175473               1.416780   
603            0.145515              0.171973               1.477203   
654            0.231547              0.304964               4.591405   
680            0.447009              0.561

In [14]:
# increasing the number of bellweathers up to 1287, to give the dataset a roughly 30 - 70 split

df1_repeated = pd.concat([df1]*22, ignore_index=True)

In [15]:
df1_repeated.head()

bellweather?  poverty_rate     population  persons_in_poverty  \
0             1     14.616667  103085.000000        13997.000000   
1             1     12.916667   17383.833333         2279.666667   
2             1     11.500000   22077.500000         2520.500000   
3             1     14.450000   19146.666667         2792.666667   
4             1     15.516667  113672.666667        17493.833333   

   rural_urban_code  urban_influence_code  metro  civilian_labor_force  \
0                 1                     1      1             459611.30   
1                 4                     4      0              29003.50   
2                 9                    12      0               1693.25   
3                 9                     8      0               2716.25   
4                 1                     1      1              56099.60   

    employed  unemployed  unemployment_rate  no_diploma  diploma_only  \
0  432442.40    27168.90              5.875    129100.8      164385.6   
1   27208.45     1794.45              5.750      1708.2        2046.2   
2    1546.15      147.10              8.680      3328.0        5596.4   
3    2544.95      171.30              6.310      1500.6        2108.2   
4   53041.15     3058.45              5.465      6537.2       11282.8   

   some_college/associate_degree  completed_college/bachelor_degree  \
0                       109497.2                           142428.6   
1                         1377.4                              668.0   
2                         3031.2                             1901.8   
3                          968.0                              553.6   
4                         4837.8                             2491.0   

   percent_no_diploma  percent_diploma_only  \
0               24.92                 30.40   
1               31.50                 35.06   
2               23.78                 40.34   
3               29.32                 41.10   
4               26.40                 44.92   

   percent_some_college/associate_degree  \
0                                  19.48   
1                                  22.38   
2                                  22.08   
3                                  18.82   
4                                  18.88   

   percent_completed_college/bachelor_degree  percent_white_male  \
0                                      23.84           33.904257   
1                                       9.36           48.651459   
2                                      12.86           49.193051   
3                                      10.24           46.399729   
4                                       9.02           43.183962   

   percent_white_female  percent_black_male  percent_black_female  \
0             35.593001           11.462633             13.033701   
1             48.761942            0.571911              0.279457   
2             48.999030            0.286621              0.185202   
3             49.539730            1.101259              0.745468   
4             46.553526            3.391660              3.620263   

   percent_native_american_male  percent_native_american_female  \
0                      0.341322                        0.357957   
1                      0.188471                        0.162475   
2                      0.092601                        0.110239   
3                      0.180719                        0.129892   
4                      0.133423                        0.137672   

   percent_asian_male  percent_asian_female  percent_hispanic_male  \
0            0.865011              1.233442               2.821145   
1            0.175473              0.175473               1.416780   
2            0.145515              0.171973               1.477203   
3            0.231547              0.304964               4.591405   
4            0.447009              0.561736               0.919513   

   percent_hispanic_female  cumulative_confirmed_cases  cumulative_deaths  
0          

In [16]:
# take all of the non-bellweathers and store them in df2

df2 = df[df["bellweather?"] == 0]

In [17]:
df2["bellweather?"].value_counts()

0    3046
Name: bellweather?, dtype: int64

In [18]:
df_new = df1_repeated.append(df2)

In [19]:
df_new["bellweather?"].value_counts()

0    3046
1    1364
Name: bellweather?, dtype: int64

In [20]:
len(df_new)

4410

In [21]:
# store all features except wins in x_new
# store wins in y_new

x_new = df_new.loc[:, df_new.columns != 'bellweather?']
y_new = df_new["bellweather?"]

In [22]:
x_new.head()

poverty_rate     population  persons_in_poverty  rural_urban_code  \
0     14.616667  103085.000000        13997.000000                 1   
1     12.916667   17383.833333         2279.666667                 4   
2     11.500000   22077.500000         2520.500000                 9   
3     14.450000   19146.666667         2792.666667                 9   
4     15.516667  113672.666667        17493.833333                 1   

   urban_influence_code  metro  civilian_labor_force   employed  unemployed  \
0                     1      1             459611.30  432442.40    27168.90   
1                     4      0              29003.50   27208.45     1794.45   
2                    12      0               1693.25    1546.15      147.10   
3                     8      0               2716.25    2544.95      171.30   
4                     1      1              56099.60   53041.15     3058.45   

   unemployment_rate  no_diploma  diploma_only  some_college/associate_degree  \
0              5.875    129100.8      164385.6                       109497.2   
1              5.750      1708.2        2046.2                         1377.4   
2              8.680      3328.0        5596.4                         3031.2   
3              6.310      1500.6        2108.2                          968.0   
4              5.465      6537.2       11282.8                         4837.8   

   completed_college/bachelor_degree  percent_no_diploma  \
0                           142428.6               24.92   
1                              668.0               31.50   
2                             1901.8               23.78   
3                              553.6               29.32   
4                             2491.0               26.40   

   percent_diploma_only  percent_some_college/associate_degree  \
0                 30.40                                  19.48   
1                 35.06                                  22.38   
2                 40.34                                  22.08   
3                 41.10                                  18.82   
4                 44.92                                  18.88   

   percent_completed_college/bachelor_degree  percent_white_male  \
0                                      23.84           33.904257   
1                                       9.36           48.651459   
2                                      12.86           49.193051   
3                                      10.24           46.399729   
4                                       9.02           43.183962   

   percent_white_female  percent_black_male  percent_black_female  \
0             35.593001           11.462633             13.033701   
1             48.761942            0.571911              0.279457   
2             48.999030            0.286621              0.185202   
3             49.539730            1.101259              0.745468   
4             46.553526            3.391660              3.620263   

   percent_native_american_male  percent_native_american_female  \
0                      0.341322                        0.357957   
1                      0.188471                        0.162475   
2                      0.092601                        0.110239   
3                      0.180719                        0.129892   
4                      0.133423                        0.137672   

   percent_asian_male  percent_asian_female  percent_hispanic_male  \
0            0.865011              1.233442               2.821145   
1            0.175473              0.175473               1.416780   
2            0.145515              0.171973               1.477203   
3            0.231547              0.304964               4.591405   
4            0.447009              0.561736               0.919513   

   percent_hispanic_female  cumulative_confirmed_cases  cumulative_deaths  
0                 2.936972                     1582465              40781  
1                 1.423279                      18

In [23]:
# approx 3% bellwethers and 70% non bellwethers
y_new.value_counts()

0    3046
1    1364
Name: bellweather?, dtype: int64

### Using SVC 
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [24]:
# split data
x_train_new, x_test_new, y_train_new, y_test_new = \
    train_test_split(x_new, y_new, test_size=0.25)

np.unique(y_new)

array([0, 1], dtype=int64)

In [25]:
# fit model (default parameters)
svclassifier = SVC(kernel='rbf')
svclassifier.fit(x_train_new, y_train_new)

SVC()

In [26]:
y_pred_class = svclassifier.predict(x_train_new)
training_accuracy = metrics.accuracy_score(y_train_new, y_pred_class)
training_error = 1 - training_accuracy
print("SVM Training set accuracy: {0}, Training set error: {1}".format(training_accuracy, training_error))

y_pred_class = svclassifier.predict(x_test_new)
testing_accuracy = metrics.accuracy_score(y_test_new, y_pred_class)
testing_error = 1 - testing_accuracy
print("SVM Test accuracy: {0}, Test error: {1}".format(testing_accuracy, testing_error))

SVM Training set accuracy: 0.6939824614454189, Training set error: 0.30601753855458114
SVM Test accuracy: 0.6808703535811423, Test error: 0.31912964641885766


In [27]:
from sklearn.metrics import classification_report

y_pred_new = svclassifier.predict(x_test_new)
print(classification_report(y_test_new, y_pred_new))

              precision    recall  f1-score   support

           0       0.73      1.00      0.84       801
           1       0.00      0.00      0.00       302

    accuracy                           0.73      1103
   macro avg       0.36      0.50      0.42      1103
weighted avg       0.53      0.73      0.61      1103



d:\users\user1\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
support_vectors = svclassifier.support_vectors_
print(support_vectors)

[[1.59500000e+01 1.47881333e+05 2.28255000e+04 ... 3.77133426e+00
  9.12578000e+05 1.44950000e+04]
 [1.38500000e+01 8.85416667e+04 1.17036667e+04 ... 6.54157835e+00
  1.19054500e+06 1.61110000e+04]
 [1.13333333e+01 3.14816667e+04 3.54166667e+03 ... 4.90650384e-01
  7.53360000e+04 9.84000000e+02]
 ...
 [8.06666667e+00 2.93013667e+05 2.22775000e+04 ... 2.62676526e+00
  2.91598300e+06 1.00341000e+05]
 [1.55166667e+01 1.13672667e+05 1.74938333e+04 ... 8.54926023e-01
  1.13209000e+06 2.64380000e+04]
 [2.28333333e+01 6.74266667e+03 1.49416667e+03 ... 7.73764680e+00
  5.81010000e+04 2.48000000e+03]]


### Run SVC Classifier 100 times

In [54]:
# store all features except wins for each sample in x_average dict
# store wins for each sample in y_average dict
x_average = {}
y_average = {}
for x in range(100):
    x_average[x] = df_new.loc[:, df_new.columns != 'bellweather?']
    y_average[x] = df_new["bellweather?"]

In [55]:
x_train_average = {}
x_test_average = {}
y_train_average = {}
y_test_average = {}
classifier_average = {}
y_pred_average = {}
sum = 0

for x in range(100):
    # split each sample into 75% training and 25% testing
    x_train_average[x], x_test_average[x], y_train_average[x], y_test_average[x] = \
        train_test_split(x_average[x], y_average[x], test_size=0.25,random_state=0)
    
    classifier_average[x] = SVC(kernel='rbf')
    
    classifier_average[x].fit(x_train_average[x], y_train_average[x])
    
    x_temp = x_average[x]

    y_pred_average[x] = classifier_average[x].predict(x_test_average[x])
    (y_test_average[x] - y_pred_average[x]).map(lambda x_temp: x_temp*x_temp).sum()
    y = metrics.accuracy_score(y_test_average[x], y_pred_average[x])
    sum += y
    
    # print average accuracy over 100 runs
    print(sum/100)

0.0068993653671804165
0.013798730734360833
0.020698096101541247
0.027597461468721666
0.034496826835902085
0.04139619220308251
0.04829555757026292
0.055194922937443346
0.06209428830462376
0.06899365367180418
0.0758930190389846
0.08279238440616501
0.08969174977334543
0.09659111514052585
0.10349048050770626
0.11038984587488669
0.1172892112420671
0.12418857660924752
0.13108794197642795
0.13798730734360837
0.14488667271078878
0.1517860380779692
0.15868540344514961
0.16558476881233003
0.17248413417951042
0.17938349954669083
0.18628286491387122
0.19318223028105164
0.20008159564823202
0.2069809610154124
0.21388032638259283
0.22077969174977322
0.22767905711695363
0.23457842248413402
0.24147778785131443
0.24837715321849482
0.25527651858567524
0.2621758839528556
0.269075249320036
0.27597461468721646
0.28287398005439685
0.28977334542157723
0.2966727107887576
0.30357207615593806
0.31047144152311845
0.31737080689029884
0.32427017225747923
0.33116953762465967
0.3380689029918401
0.34496826835902056
0.

In [56]:
# print average accuracy over 100 runs

print(sum/100)

#training_accuracy = metrics.accuracy_score(y_train_new, y_pred_class)

0.6899365367180418


### Define Functions

In [57]:
def run_100():
    x_average = {}
    y_average = {}
    for x in range(100):
        x_average[x] = df_new.loc[:, df_new.columns != 'bellweather?']
        y_average[x] = df_new["bellweather?"]

    x_train_average = {}
    x_test_average = {}
    y_train_average = {}
    y_test_average = {}
    classifier_average = {}
    y_pred_average = {}
    sum = 0

    for x in range(100):
        # split each sample into 75% training and 25% testing
        x_train_average[x], x_test_average[x], y_train_average[x], y_test_average[x] = \
            train_test_split(x_average[x], y_average[x], test_size=0.25,random_state=0)

        classifier_average[x] = SVC(kernel='rbf')

        classifier_average[x].fit(x_train_average[x], y_train_average[x])

        x_temp = x_average[x]

        y_pred_average[x] = classifier_average[x].predict(x_test_average[x])
        (y_test_average[x] - y_pred_average[x]).map(lambda x_temp: x_temp*x_temp).sum()
        y = metrics.accuracy_score(y_test_average[x], y_pred_average[x])
        sum += y

    # print average accuracy over 100 runs
    print(sum/100)

In [58]:
def svm_accuracy(clf):
    y_pred_class = clf.predict(x_train_new)
    training_accuracy = metrics.accuracy_score(y_train_new, y_pred_class)
    training_error = 1 - training_accuracy
    print("SVM Training set accuracy: {0}, Training set error: {1}".format(training_accuracy, training_error))
    y_pred_class = clf.predict(x_test_new)
    testing_accuracy = metrics.accuracy_score(y_test_new, y_pred_class)
    testing_error = 1 - testing_accuracy
    print("SVM Test accuracy: {0}, Test error: {1} \n".format(testing_accuracy, testing_error))

In [59]:
def svm_classification_report(clf):
    print("CLassification Report")
    y_pred_new = clf.predict(x_test_new)
    print(classification_report(y_test_new, y_pred_new))

### C value tuning

In [60]:
cs = [0.1, 1, 10, 100, 1000, 10000]
for c in cs:
    svclassifier = SVC(kernel='rbf', C=c)
    svclassifier.fit(x_train_new, y_train_new)
    print("C value of " + str(c))
    svm_accuracy(svclassifier)

C value of 0.1
SVM Training set accuracy: 0.6885394617478077, Training set error: 0.31146053825219233
SVM Test accuracy: 0.6971894832275612, Test error: 0.30281051677243875 

C value of 1
SVM Training set accuracy: 0.6885394617478077, Training set error: 0.31146053825219233
SVM Test accuracy: 0.6971894832275612, Test error: 0.30281051677243875 

C value of 10
SVM Training set accuracy: 0.6918657393407922, Training set error: 0.3081342606592078
SVM Test accuracy: 0.7017225747960109, Test error: 0.29827742520398914 

C value of 100
SVM Training set accuracy: 0.7124281826428788, Training set error: 0.28757181735712123
SVM Test accuracy: 0.7207615593834995, Test error: 0.2792384406165005 

C value of 1000
SVM Training set accuracy: 0.7178711823404899, Training set error: 0.28212881765951014
SVM Test accuracy: 0.7225747960108794, Test error: 0.27742520398912063 

C value of 10000
SVM Training set accuracy: 0.7405503477472029, Training set error: 0.25944965225279715
SVM Test accuracy: 0.7379

#### Optimal value for C
C value of 100000  
SVM Training set accuracy: 0.8771763216207661, Training set error: 0.12282367837923391  
SVM Test accuracy: 0.8766603415559773, Test error: 0.12333965844402273 

#### Results from above (takes a long time to run)
C value of 0.1  
SVM Training set accuracy: 0.6929408040519152, Training set error: 0.30705919594808484  
SVM Test accuracy: 0.7058823529411765, Test error: 0.2941176470588235 

C value of 1  
SVM Training set accuracy: 0.7059195948084837, Training set error: 0.29408040519151635  
SVM Test accuracy: 0.7201138519924098, Test error: 0.27988614800759015 

C value of 10  
SVM Training set accuracy: 0.7176321620766065, Training set error: 0.2823678379233935  
SVM Test accuracy: 0.7267552182163188, Test error: 0.27324478178368117 

C value of 100  
SVM Training set accuracy: 0.7318771763216207, Training set error: 0.26812282367837925  
SVM Test accuracy: 0.7381404174573055, Test error: 0.2618595825426945 

C value of 1000  
SVM Training set accuracy: 0.7932890155112378, Training set error: 0.20671098448876224  
SVM Test accuracy: 0.7979127134724858, Test error: 0.20208728652751418 

C value of 10000  
SVM Training set accuracy: 0.8420386198163976, Training set error: 0.1579613801836024  
SVM Test accuracy: 0.8415559772296015, Test error: 0.15844402277039848 

C value of 100000  
SVM Training set accuracy: 0.8771763216207661, Training set error: 0.12282367837923391  
SVM Test accuracy: 0.8766603415559773, Test error: 0.12333965844402273 

C value of 1000000  
SVM Training set accuracy: 0.862298195631529, Training set error: 0.13770180436847101  
SVM Test accuracy: 0.8434535104364327, Test error: 0.15654648956356731 

C value of 10000000  
SVM Training set accuracy: 0.8208293763849319, Training set error: 0.17917062361506808  
SVM Test accuracy: 0.7941176470588235, Test error: 0.20588235294117652 